In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import read_infections_and_deaths
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

from june.records import RecordReader

No --data argument given - defaulting to:
/cosma6/data/dp004/dc-cues1/JUNE-private/data
No --configs argument given - defaulting to:
/cosma6/data/dp004/dc-cues1/JUNE-private/june/configs
--------------------------------------------------------------------------
By default, for Open MPI 4.0 and later, infiniband ports on a device
are not used by default.  The intent is to use UCX for these devices.
You can override this policy by setting the btl_openib_allow_ib MCA parameter
to true.

  Local host:              login7b
  Local adapter:           mlx5_0
  Local port:              1

--------------------------------------------------------------------------
--------------------------------------------------------------------------

  Local host:   login7b
  Local device: mlx5_0
--------------------------------------------------------------------------


In [3]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

plt.style.use(['science'])
plt.style.reload_library()

In [5]:
n_runs = 8

In [6]:
DATA_DIR = Path('/cosma6/data/dp004/dc-quer1/vaccine_paper_runs/')

In [7]:
# read one people df
read = RecordReader(
    results_path=DATA_DIR / 'baselines/results/run_000'
)
people_df = read.table_to_df('population')

In [ ]:
%%time

infections_dfs, deaths_dfs = [], []
for run in range(n_runs):
    inf_df, death_df = read_infections_and_deaths(
        data_dir =DATA_DIR / 'baselines/results/', run=run
    )
    infections_dfs.append(inf_df)
    deaths_dfs.append(death_df)

In [ ]:
%%time

first_wave_infections_dfs, first_wave_deaths_dfs = [], []

for run in range(n_runs):
    inf_df, death_df = read_infections_and_deaths(run=run, 
            data_dir=Path('/cosma5/data/durham/covid19/results_kobe_best_runs/')
    )
    first_wave_infections_dfs.append(inf_df)
    first_wave_deaths_dfs.append(death_df)

In [ ]:
def average_sumaries(summary_list):
    df_concat = pd.concat(summary_list)
    by_row_index = df_concat.groupby(df_concat.index)
    df_means = by_row_index.mean()
    df_std = by_row_index.std()   
    return df_means, df_std/np.sqrt(len(summary_list))

## Age distribution deaths

In [ ]:
age_bins = np.arange(0,101,6)

n_people_by_age = people_df.groupby([pd.cut(people_df['age'],
    bins=age_bins, right=False)]).size()


In [ ]:

deaths_by_age = [
    100.*death_df.groupby(
        pd.cut(
            death_df['age'], bins=age_bins, right=False
        )).size()/len(death_df)   for death_df in deaths_dfs
]

first_wave_deaths_by_age = [
    100.*death_df.groupby(
        pd.cut(
            death_df['age'], bins=age_bins, right=False
        )).size()/len(death_df)   for death_df in first_wave_deaths_dfs
]

In [ ]:
avg_deaths_by_age, std_deaths_by_age = average_sumaries(deaths_by_age)
avg_first_wave_deaths_by_age, std_first_wave_deaths_by_age = average_sumaries(first_wave_deaths_by_age)

In [ ]:
def revert_to_string(age):
    a = age.left
    b = age.right
    return f"{a}-{b}"

avg_deaths_by_age = avg_deaths_by_age.rename(revert_to_string)
avg_first_wave_deaths_by_age = avg_first_wave_deaths_by_age.rename(revert_to_string)

In [ ]:
std_deaths_by_age = std_deaths_by_age.rename(revert_to_string)
std_first_wave_deaths_by_age = std_first_wave_deaths_by_age.rename(revert_to_string)

In [ ]:
deaths_to_plot = pd.DataFrame(index=avg_deaths_by_age.index)
deaths_to_plot.loc[:, 'Projected'] = avg_deaths_by_age
deaths_to_plot.loc[:, '1st Wave'] = avg_first_wave_deaths_by_age

std_deaths_to_plot = pd.DataFrame(index=std_deaths_by_age.index)
std_deaths_to_plot.loc[:, 'Projected'] = std_deaths_by_age
std_deaths_to_plot.loc[:, '1st Wave'] = std_first_wave_deaths_by_age

In [ ]:
fig, ax = plt.subplots()


deaths_to_plot.plot.bar(
            capsize=4,
            ylabel="",
            xlabel="Age",
            width=0.8,
            alpha=0.7,
            ax=ax,
            yerr=std_deaths_to_plot,
        )
ax.set_ylabel('Deaths in age group [\%]')
plt.savefig('figures/deaths_by_age.pdf')

## Crude fatality rates

In [ ]:

n_by_vaccine_age = people_df.groupby([
                'vaccine_type', 
                pd.cut(people_df['age'],
    bins=age_bins, right=False)]).size()

In [ ]:

def compute_crude_death_rate_by_age(n_by_vaccine_age, deaths_df, vaccine_type):
    if vaccine_type is not None:
        n_by_age = n_by_vaccine_age.loc[vaccine_type]
        death_df = deaths_df[(deaths_df['vaccine_type'] == vaccine_type)]
    else:
        n_by_age = n_by_vaccine_age.groupby(['age']).sum()
        death_df = deaths_df
    death_by_age = death_df.groupby([
            pd.cut(death_df['age'],
                bins=age_bins, right=False)]).size()
    death_rate = death_by_age / n_by_age
    death_rate = death_rate.fillna(0)
    return 100.*death_rate

In [ ]:
avg_first_wave_drs, std_first_wave_drs = average_sumaries([
    compute_crude_death_rate_by_age(n_by_vaccine_age, deaths_df,None) 
    for deaths_df in first_wave_deaths_dfs
])


avg_unvaccinated_drs, std_unvaccinated_drs = average_sumaries([
    compute_crude_death_rate_by_age(n_by_vaccine_age, deaths_df,'none') 
    for deaths_df in deaths_dfs
])

avg_vaccinated_drs, std_vaccinated_drs = average_sumaries([
    compute_crude_death_rate_by_age(n_by_vaccine_age, deaths_df,None) 
    for deaths_df in deaths_dfs
])


In [ ]:
dr_toplot = pd.DataFrame(index=avg_first_wave_drs.index)


In [ ]:
dr_toplot.loc[:,'Projected Vaccinated'] = avg_vaccinated_drs
dr_toplot.loc[:,'1st Wave'] = avg_first_wave_drs
dr_toplot.loc[:,'Projected Unvaccinated'] = avg_unvaccinated_drs


In [ ]:
std_dr_toplot = pd.DataFrame(index=avg_first_wave_drs.index)
std_dr_toplot.loc[:,'Projected Vaccinated'] = std_vaccinated_drs
std_dr_toplot.loc[:,'1st Wave'] = std_first_wave_drs
std_dr_toplot.loc[:,'Projected Unvaccinated'] = std_unvaccinated_drs


In [ ]:

dr_toplot = dr_toplot.rename(mapper=revert_to_string)

std_dr_toplot = std_dr_toplot.rename(mapper=revert_to_string)


In [ ]:
fig, ax = plt.subplots(ncols=2,sharex=False,sharey=False,figsize=(9.5,3.))


deaths_to_plot.plot.bar(
            capsize=4,
            ylabel="",
            xlabel="Age",
            width=0.8,
            alpha=0.7,
            ax=ax[0],
            yerr=std_deaths_to_plot,
        )
ax[0].set_ylabel('Deaths in age group [\%]')

dr_toplot.plot.bar(
            capsize=4,
            ylabel="Crude Fatality Rate [\%]",
            xlabel="Age",
            width=0.8,
            alpha=0.7,
            ax=ax[1],
            yerr=std_dr_toplot,
        )

ax[1].set_yscale('log')
ax[1].set_ylim(1e-3)

plt.savefig('figures/crude_fatality_rates.pdf')

In [ ]:
fig, ax = plt.subplots(ncols=2,sharex=False,sharey=False,figsize=(9.5,3.))


deaths_to_plot.plot.bar(
            capsize=4,
            ylabel="",
            xlabel="Age",
            width=0.8,
            alpha=0.7,
            ax=ax[0],
            yerr=std_deaths_to_plot,
        )
ax[0].set_ylabel('Deaths in age group [\%]')

dr_toplot.plot.bar(
            capsize=4,
            ylabel="Crude Fatality Rate [\%]",
            xlabel="Age",
            width=0.8,
            alpha=0.7,
            ax=ax[1],
            yerr=std_dr_toplot,
        )

plt.savefig('figures/linear_crude_fatality_rates.pdf')